In [1]:
import os
import pandas as pd
import numpy as np
import openai
import faiss
from dotenv import load_dotenv
load_dotenv()

from sentence_transformers import SentenceTransformer

In [2]:
os.getcwd()

'C:\\Users\\yaobv\\hanson_app\\notebooks'

In [3]:
openai.organization = os.environ.get('OPENAI_ORG')
openai.api_key = os.environ.get('OPENAI_KEY')

In [5]:
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

df = pd.read_csv('../data/hanson_df.csv')
embeddings = np.load('../data/embedding_array.npy')

posts = df['body'].values
titles = df['title'].values
links = df['url'].values

embeddings_idx = faiss.IndexFlatL2(embeddings.shape[1])
embeddings_idx.add(embeddings)

In [43]:
def get_hanson_articles(query=None, top_k=5):
    """    
    a function to return the body of the top match and the blog title
    of the top 5 entries (for now, testing)
    """
    
    query_embedding = model.encode(query)
    
    d, i =  embeddings_idx.search(np.array([query_embedding]), top_k)

    post_bodies = [posts[x] for x in i[0]]
    post_titles = [titles[x] for x in i[0]]
    links_ = [links[x] for x in i[0]]
    
    return post_bodies[0] + post_bodies[1], links_[0]

In [67]:
messages = [
    {"role": "system", "content": "You're analytical and kind. You want to help the user understand the economist Robin Hanson's worldview."},

]

In [70]:
def hanson_chat_test(query_test=None):    
    
    posts, link = get_hanson_articles(query=query_test, top_k=2)
    prompt = messages.copy()
    
    prompt.append({"role" : "user", "content" : f"I would like you to consider the following before answering the question: '{posts}'. Note that if the foregoing doesn't seem relevant, please inform the user that Hanson doesnt address that on his blog. Stick to what Hanson would say. {query_test}"})
    
    response = (
        openai
        .ChatCompletion
        .create(
            model="gpt-3.5-turbo",
            max_tokens=300,
            messages=prompt)
        )
    print(response.usage.completion_tokens)

    generated_text = response.choices[0].message.content
    
    messages.append({"role" : "user", "content" : f'{query_test}'})
    messages.append({"role" : "assistant", "content" : generated_text})
    
    return generated_text, link

In [71]:
hanson_chat_test("Would Hanson say the ancients were most socially desirable, or contemporary humans?")

104


("Based on Robin Hanson's beliefs about the forager/farmer thesis, he would likely argue that the mating habits of contemporary humans are a reflection of our ancient forager ancestors. Thus, he would suggest that the characteristics that were most socially desirable in ancient times are still desirable now, such as physical attractiveness and social status. However, Hanson would also acknowledge that modern humans have evolved and may have different preferences and values, but he would argue that the forager way of life is still influential in shaping our mating behavior.",
 'https://www.overcomingbias.com/p/new-paleolithic-matinghtml')

In [62]:
sum([len(x['content']) for x in messages]) / 6

2122.5

In [76]:
if len(messages) > 1:
    new_messages = [messages[0]]
    new_messages.append(messages[-1])
    print(new_messages)

[{'role': 'system', 'content': "You're analytical and kind. You want to help the user understand the economist Robin Hanson's worldview."}, {'role': 'assistant', 'content': "Based on Robin Hanson's beliefs about the forager/farmer thesis, he would likely argue that the mating habits of contemporary humans are a reflection of our ancient forager ancestors. Thus, he would suggest that the characteristics that were most socially desirable in ancient times are still desirable now, such as physical attractiveness and social status. However, Hanson would also acknowledge that modern humans have evolved and may have different preferences and values, but he would argue that the forager way of life is still influential in shaping our mating behavior."}]
